In [1]:
from torch import nn, load
from resnet import ResNet

In [2]:
class ResNetEnsembleInfraredRGB(nn.Module):

    def __init__(self, num_classes, ResNetRGB, ResNetIR):

        super().__init__()
        print(ResNetRGB)
        
        self.ResNetRGB = ResNetRGB
        self.ResNetIR = ResNetIR

        
        for param in self.ResNetRGB.parameters(): # Freeze all parameters
            param.requires_grad = False  
        
        for param in self.ResNetRGB.model.layer4.parameters():
            param.requires_grad = True
        
        for param in self.ResNetRGB.model.layer3.parameters():
            param.requires_grad = True

        
        for param in self.ResNetIR.parameters(): # Freeze all parameters
            param.requires_grad = False  
        
        #ResNetIR layer for will not be used
        
        for param in self.ResNetRGB.model.layer3.parameters():
            param.requires_grad = True

            
            
        self.upsample = n.Sequential( #Double 3 times
                                        nn.ConvTranspose2d(256, 256, 3, stride=2, padding=1),
                                        nn.ConvTranspose2d(256, 256, 3, stride=2, padding=1),
                                        nn.ConvTranspose2d(256, 256, 3, stride=2, padding=1),
                                    )

        
        #After concat of layers from infrared and rgb. do 1x1 conv layer to reduce dimensions.
        self.model_Head = nn.Sequential(
                                    nn.Conv2d(512, 256, kernel_size=1, stride=1, padding=0,bias=False),
                                    ResNetRGB.model.layer4,
                                    ResNetRGB.model.avgpool,
                                    nn.Linear( 512 , num_classes ) 
                                    )
        
        

    
    
        
    
    def forward ( self , x_rgb, x_infrared): 
        
        #rgb_base
        x_rgb = self.ResNetRGB.model.conv1(x_rgb)
        x_rgb = self.ResNetRGB.model.bn1(x_rgb)
        x_rgb = self.ResNetRGB.model.relu(x_rgb)
        x_rgb = self.ResNetRGB.model.maxpool(x_rgb)
        x_rgb = self.ResNetRGB.model.layer1(x_rgb)
        x_rgb = self.ResNetRGB.model.layer2(x_rgb)
        x_rgb = self.ResNetRGB.model.layer3(x_rgb)
        
        
        #infrared_base
        x_infrared = self.ResNetIR.model.conv1(x_infrared)
        x_infrared = self.ResNetIR.model.bn1(x_infrared)
        x_infrared = self.ResNetIR.model.relu(x_infrared)
        x_infrared = self.ResNetIR.model.maxpool(x_infrared)
        x_infrared = self.ResNetIR.model.layer1(x_infrared)
        x_infrared = self.ResNetIR.model.layer2(x_infrared)
        x_infrared = self.ResNetIR.model.layer3(x_infrared)
        
        # give outputs the same dimension... scale up infrared or downscale rgb
        x_infrared= self.upsample(x_infrared)

        
        x = torch.cat((xRGB, xIR), dim=1)
        x = self.model_Head(x)

        return x

In [3]:

time_stamp_rgb = '20200206_1907'
model_path_rgb = './Work_dirs/work_dirs_external/rgb/' + time_stamp_rgb +'/model_best.pth.tar'
model_rgb = ResNet(image_channels=3, num_classes=9)
model_rgb.load_state_dict(load(model_path_rgb)['state_dict'])

time_stamp_infrared = '20200206_1346'
model_path_infrared = './Work_dirs/work_dirs_external/infrared/' + time_stamp_infrared  +'/model_best.pth.tar'
model_infrared = ResNet(image_channels=3, num_classes=9)
model_infrared.load_state_dict(load(model_path_infrared)['state_dict'])

model = ResNetEnsembleInfraredRGB( 9, model_rgb, model_infrared)



ResNet(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

NameError: name 'n' is not defined